In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon

from scipy.spatial import cKDTree
import haversine as hs  # pip install haversine

#see what kind of data is located in the file wohnungsregister and see if i can do a match on the address or if the 

In [4]:
gdf_register = gpd.read_file(r'C:\Users\gigim\Documents\GitHub\mlp\gedodata_backup\\data_Wohnungsregister.gpkg')

In [7]:
#convert geoDataFrame to DF
def todf(gdf):
    # Transform to WGS 84 CRS
    gdf = gdf.to_crs(epsg=4326)
    # First ensure that your geometries are points
    assert all(gdf.geometry.geom_type == 'Point'), "All geometries must be Points."

    # Create longitude and latitude columns from the geometry
    gdf['longitude'] = gdf.geometry.x
    gdf['latitude'] = gdf.geometry.y

    # Now convert your GeoDataFrame to a DataFrame
    df = pd.DataFrame(gdf.drop(columns='geometry'))
    return df

df_register = todf(gdf_register)

In [36]:
def merge_address(df):
    # Use the apply function to create a new 'address' column
    df['address'] = df.apply(
        lambda row: f"{row['strname']} {row['deinr']}, {row['dplz4']} {row['dplzname']}",
        axis=1
    )
    return df

# Example usage
df_register_1 = merge_address(df_register)

In [46]:
# read csv file of addresses with long and lat
df_24 = pd.read_csv('immoscout24_geocoded.csv', delimiter='|')

In [47]:
# Replace 'str.' with 'strasse' in the 'address' column
df_24['address_x'] = df_24['address_x'].str.replace('str\.', 'strasse', regex=True)


In [26]:
df_register_1.columns

Index(['egid', 'ewid', 'edid', 'whgnr', 'wstwk', 'wstwklang', 'wbez', 'wmehrg',
       'wmehrglang', 'wbauj', 'wabbj', 'wstat', 'wstatlang', 'warea', 'wazim',
       'wkche', 'wkchelang', 'dkode', 'dkodn', 'deinr', 'strname', 'dplz4',
       'dplzname', 'dexpdat', 'objectid', 'longitude', 'latitude', 'addresse'],
      dtype='object')

In [41]:
# Columns to drop
columns_to_drop = ['egid', 'ewid', 'edid', 'whgnr', 'wstwk', 'wstwklang', 'wbez', 'wmehrg',
       'wmehrglang', 'wstat', 'wstatlang', 'warea', 'wabbj', 'wazim',
       'wkche', 'wkchelang', 'dkode', 'dkodn', 'deinr', 'strname', 'dplz4',
       'dplzname', 'dexpdat', 'objectid', 'longitude', 'latitude', 'addresse']

# Dropping the columns
df_register_2 = df_register_1.drop(columns=columns_to_drop)

In [50]:
df_register_2.to_csv('geodata/df_register.csv')

In [48]:
# Merge df1 and df2 on the 'address' of df1 and 'address_x' of df2
merged_df = pd.merge(df_24, df_register_2, left_on='address_x', right_on='address', how='left')

In [49]:
merged_df.to_clipboard()

In [53]:
df_register_3 = pd.read_csv('geodata/df_register.csv')
df_register_3

,wbauj,address
0,1999.0,"Hofstrasse 64, 8032 Zürich"
1,1999.0,"Hofstrasse 64, 8032 Zürich"
2,1999.0,"Hofstrasse 64, 8032 Zürich"
3,1999.0,"Hofstrasse 64, 8032 Zürich"
4,1999.0,"Hofstrasse 64, 8032 Zürich"
...,...,...
267546,2015.0,"Brandschenkestrasse 64, 8002 Zürich"
267547,2015.0,"Brandschenkestrasse 64, 8002 Zürich"
267548,2015.0,"Brandschenkestrasse 66, 8002 Zürich"
267549,2015.0,"Brandschenkestrasse 66, 8002 Zürich"
